NO TENGO LAS BASES PARA CORRERLAS, ESTAN EN MECON

In [2]:
##############      Configuración      ##############
import os
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict
from PIL import Image
from dotenv import dotenv_values

pd.set_option('display.max_columns', None)
env = dotenv_values("globals.env")

path_proyecto   = env['PATH_PROYECTO']    
path_datain     = env['PATH_DATAIN']
path_dataout    = env['PATH_DATAOUT']
path_scripts    = env['PATH_SCRIPTS']    
path_satelites  = env['PATH_SATELITES']
path_logs       = env['PATH_LOGS']     
path_outputs    = env['PATH_OUTPUTS']     
# path_programas  = globales[7]

In [3]:
import affine
import geopandas as gpd
import rasterio.features
import xarray as xr
import shapely.geometry as sg
import pandas as pd 

def polygonize(da: xr.DataArray) -> gpd.GeoDataFrame:
    """
    Polygonize a 2D-DataArray into a GeoDataFrame of polygons.

    Parameters
    ----------
    da : xr.DataArray

    Returns
    -------
    polygonized : geopandas.GeoDataFrame
    """
    if da.dims != ("y", "x"):
        raise ValueError('Dimensions must be ("y", "x")')

    values = da.values
    transform = da.attrs.get("transform", None)
    if transform is None:
        raise ValueError("transform is required in da.attrs")
    transform = affine.Affine(*transform)
    shapes = rasterio.features.shapes(values, transform=transform)

    geometries = []
    colvalues = []
    for (geom, colval) in shapes:
        geometries.append(sg.Polygon(geom["coordinates"][0]))
        colvalues.append(colval)

    gdf = gpd.GeoDataFrame({"value": colvalues, "geometry": geometries})
    gdf.crs = da.attrs.get("crs")
    return gdf


In [7]:
urbanData = xr.open_dataset(f'{path_datain}\WSF 2015 - Urbanizacion\WSF 2015 AMBA.tif')

'D:\\Maestría\\Tesis\\Repo\\data\\data_in\\WSF 2015 AMBA.tif'

In [8]:
# Load data
urbanData = xr.open_dataset(f'{path_datain}\WSF 2015 - Urbanizacion\WSF 2015 AMBA.tif').squeeze('band', drop=True)
icpag = gpd.read_file(
    r"R:\Shapefiles\ICPAG\Sin barrios pop y cerr\Aglomerados de mas de 500k habitantes\base_icpag_500k.shp"
)
collapse_link = pd.read_stata(r"D:\MECON\7. Ingreso Esperado\data\data_out\7.10 Small Area Estimation (Nico)\predict_ingreso_collapse.dta")

DriverError: R:\Shapefiles\ICPAG\Sin barrios pop y cerr\Aglomerados de mas de 500k habitantes\base_icpag_500k.shp: No such file or directory

In [ ]:
# Merge icpag indicators with ELL estimation
icpag['link'] = icpag['link'].astype(str).str.zfill(9)
collapse_link['link'] = collapse_link['link'].astype(str).str.zfill(9)

icpag = icpag.merge(collapse_link, on='link', how='left', validate='1:1')

In [ ]:
# Get a single shape of the AMBA
icpag = icpag.to_crs(4326)
amba_polygon = icpag[icpag.AMBA_legal == 1].dissolve().geometry
bounds = amba_polygon.bounds

# Polygonize the urban data and keep only the AMBA
wsf2015 = polygonize(urbanData)
wsf2015 = wsf2015.to_crs(4326)

minx=bounds.minx.values[0]
maxx=bounds.maxx.values[0]
miny=bounds.miny.values[0]
maxy=bounds.maxy.values[0]

# Clip geodataframe based on bounding box
wsf2015_amba = wsf2015.cx[minx:maxx, miny:maxy]

# Keep only urban areas from wsf
wsf2015_amba = wsf2015_amba[wsf2015_amba.value != 0]
wsf2015_amba['urban'] = True
wsf2015_amba = wsf2015_amba.drop(columns='value')

# Crop census tracts based on urban wsd2015
wsf2015_amba = wsf2015.clip(amba_polygon.values[0])

In [ ]:
# Keep only urban areas from wsf
wsf2015_amba = wsf2015_amba[wsf2015_amba.value != 0]
wsf2015_amba['urban'] = True
wsf2015_amba = wsf2015_amba.drop(columns='value')

# Crop esf2015 based on shape of AMBA
wsf2015_amba = wsf2015_amba.clip(amba_polygon.values[0])

In [ ]:
wsf2015_amba.plot()

In [ ]:
# Save shapefile
wsf2015_amba_poly = wsf2015_amba.dissolve()
wsf2015_amba_poly.to_file(f'{path_dataout}\wsf2015_amba.shp')

In [ ]:
# Set wsf in meter crs for Argentina (Gauss-Kruger zone 5 - faja 5)
#   see: https://www.ign.gob.ar/NuestrasActividades/ProduccionCartografica/sistemas-de-proyeccion
wsf2015_amba_poly = wsf2015_amba_poly.to_crs(5348)
icpag = icpag.to_crs(5348)
amba_polygon = amba_polygon.to_crs(5348)

# Clip census tracts based on urban/rural wsd2015
urban_mask = wsf2015_amba_poly
urban = icpag[icpag.AMBA_legal == 1].clip(urban_mask)
urban['urban'] = True

rural_mask = amba_polygon.difference(wsf2015_amba_poly.buffer(100)) # 1-urban_mask
rural = icpag[icpag.AMBA_legal == 1].clip(rural_mask)
rural['urban'] = False

both = pd.concat([urban, rural])

In [ ]:
# Update data for rural areas
indicator_cols = [
    'rmin',
    'rmax',
    'rminp50',
    'rminp25',
    'rminp75',
    'rmaxp50',
    'rmaxp25',
    'rmaxp75',
    'p_usd',
    'pm2',
    'obs_pm2',
    'pm2_2019',
    'obs_pm2_19',
    'p_usd_2019',
    'personas',
    'viv_part',
    'icv2010',
    'nbi_rc',
]
for col in indicator_cols:
    both.loc[both.urban == False, col] = 0

both.to_file(f'{path_dataout}\census_tracts_with_indicators.shp')
both.to_feather(f'{path_dataout}\census_tracts_with_indicators.feather')

In [ ]:
import geopandas as gpd
df = gpd.read_feather(f'{path_dataout}\census_tracts_with_indicators.feather')

In [ ]:
df[(df.link == "061190403") & (df.urban == True)].explore(tiles="https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="asd")

In [ ]:
both['urban'] = both['urban'].astype(int)
both.explore(column='urban')